In [1]:
import os
from glob import glob
import csv
from music21 import *
from music21.features import jSymbolic as js

## Settings

In [2]:
# Classical music target and destination
# corpus_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Feature Extraction\Classical\Classical Extracted Notes'
# export_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Feature Extraction\Classical\Classical Features.csv'

# Jazz music target and destination
# corpus_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Feature Extraction\Jazz\Jazz Extracted Notes'
# export_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Feature Extraction\Jazz\Jazz Features.csv'

# Generated classical music target and destination
# corpus_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Generation\Classical\Generated Songs'
# export_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Generation\Classical\Features'

# Generated jazz music target and destination
corpus_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Generation\Jazz\Generated Songs'
export_path = r'C:\Users\alext\Desktop\School\2021 Spring\CS 271\Final Project\Generation\Jazz\Features'

# File and folder naming format for generated songs
file_format = 'epochs_40_samp_length_{}_seq_length_{}'
# Song generation parameters
sample_legths = [1000, 2000, 3000]
sequence_lengths = [10, 50, 100, 200, 300]

# The duration of each note
note_duration = 1

# The list of features to extract
# Refer to the JSymbolic docs page for explanations on each
# https://web.mit.edu/music21/doc/moduleReference/moduleFeaturesJSymbolic.html
feature_extractors = [
    js.AmountOfArpeggiationFeature,
    js.AverageMelodicIntervalFeature,
    js.ChromaticMotionFeature,
    js.DirectionOfMotionFeature,
    js.DistanceBetweenMostCommonMelodicIntervalsFeature,
    js.DurationOfMelodicArcsFeature,
    js.ImportanceOfBassRegisterFeature,
    js.ImportanceOfHighRegisterFeature,
    js.ImportanceOfMiddleRegisterFeature,
    js.IntervalBetweenStrongestPitchClassesFeature,
    js.IntervalBetweenStrongestPitchesFeature,
    js.MelodicFifthsFeature,
    js.MelodicOctavesFeature,
    js.MelodicThirdsFeature,
    js.MelodicTritonesFeature,
    js.MostCommonMelodicIntervalFeature,
    js.MostCommonMelodicIntervalPrevalenceFeature,
    js.MostCommonPitchClassPrevalenceFeature,
    js.MostCommonPitchPrevalenceFeature,
    js.NumberOfCommonMelodicIntervalsFeature,
    js.NumberOfCommonPitchesFeature,
    js.PitchClassVarietyFeature,
    js.PitchVarietyFeature,
    js.PrimaryRegisterFeature,
    js.RangeFeature,
    js.RelativeStrengthOfTopPitchClassesFeature,
    js.RelativeStrengthOfTopPitchesFeature,
    js.RepeatedNotesFeature,
    js.SizeOfMelodicArcsFeature,
    js.StepwiseMotionFeature
]

## Convert a CSV File to a Stream

In [3]:
# Convert a csv file back to a music21 stream object
def csv_to_stream(csv_file):
    # A list of the notes in the song
    note_list = []
    # The offset of the current note
    offset = 0

    # Open the csv file
    with open(csv_file, 'r') as csvfile:
        datareader = csv.reader(csvfile)
        
        # Iterate over each row
        for row in datareader:
            if row:
                # Append the notes to the stream
                append_notes(row, offset, note_list)
                # Increase the offset
                offset += note_duration
    
    # Create a music21 stream object from the list
    return stream.Stream(note_list)


# Add a list of note pitches to the given list
def append_notes(pitch_list, offset, note_list):
    phrase = []
    
    # Convert all the pitches into music21 note objects
    for pitch in pitch_list:
        new_note = note.Note(int(pitch))
        new_note.offset = offset
        phrase.append(new_note)
    
    # If there is only oen note, append it as-is to the song
    if len(phrase) == 1:
        note_list.append(phrase[0])
    
    # If there are multiple notes, organize them into a chord
    else:
        new_chord = chord.Chord(phrase)
        note_list.append(new_chord)

## Extract the Features From a File

In [4]:
def extract_features(song_stream):
    data_set = features.DataSet(classLabel='Genre')
    
    data_set.addData(song_stream)
    data_set.addFeatureExtractors(feature_extractors)
    data_set.process()
    
    return data_set.getFeaturesAsList(includeClassLabel=False, includeId=False)

## Extract the Features From a Corpus

In [5]:
def extract_corpus():
    # Go to the corpus directory
    os.chdir(corpus_path)
    
    # Count the files processed
    row = 0

    # Write the feature to a csv file
    with open(export_path, 'w', newline='') as csv_file:
        write = csv.writer(csv_file)

        # Export the notes of each score as a csv file
        for song_file in glob('*.csv'):
            row += 1

            if row % 50 == 0:
                print(row)

            # Extract the features of the song
            song_stream = csv_to_stream(song_file)
            exctracted_features = extract_features(song_stream)

            # Write the features to the csv file
            write.writerows([exctracted_features])

## Extract the Features From Generated Songs

In [6]:
def extract_generated(file_format):
    # Iterate through the parameter combinations
    for sample_length in sample_legths:
        for sequence_length in sequence_lengths:
            file_name = file_format.format(sample_length, sequence_length)
            # Construct the corpus and export paths
            curr_corpus_path = os.path.join(corpus_path, file_name)
            curr_export_path = os.path.join(export_path, file_name + '.csv')
            
            # Go to the corpus directory
            os.chdir(curr_corpus_path)

            # Count the files processed
            row = 0

            # Write the feature to a csv file
            with open(curr_export_path, 'w', newline='') as csv_file:
                write = csv.writer(csv_file)

                # Export the notes of each score as a csv file
                for song_file in glob('*.csv'):
                    row += 1

                    if row % 50 == 0:
                        print(row)

                    # Extract the features of the song
                    song_stream = csv_to_stream(song_file)
                    exctracted_features = extract_features(song_stream)

                    # Write the features to the csv file
                    write.writerows([exctracted_features])

## Driver

In [7]:
print('Extracting...\n')

# extract_corpus()
extract_generated(file_format)

print('\nFinished')

Extracting...


Finished
